# Applied Machine Learning 



## NLP for NLU using Transformers

#### This is the homework for Applied Machine Learning, Part 2, NLP for NLU.
##### Week -3




<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> Part 1: Summarization and Topics Modeling of Articles</h2>
</div>


1. Read large text:
  -- Google search for a topic 
  -- get list of top ten articles
2. extract into sentences ended with '.' [period]
3. write to a text file
4. read text file into list
5. summarize text using transformers 
   
   Find summaries -- see project details at end of notebook 
   
   try and plot comparisons of 5 combinations/permutations of the list sequence created in step 4 above
   to see the effects of chunking on the summarization
6. Do 2 distillations on the original article and the summary:
   topics, named entity recognition

## Scraping the top 3 urls from google search for topic Thanksgiving

In [28]:
import urllib
from bs4 import BeautifulSoup
import requests
import webbrowser

text = 'fox news covid'
text = urllib.parse.quote_plus(text)

url = 'https://google.com/search?q=' + text

response = requests.get(url)
new_urls=set()
soup = BeautifulSoup(response.text, "html.parser")
for g in soup.find_all('a',href=True):
  if 'foxnews.com' in g['href']:
    new_urls.add(g['href'][7:])
    if len(new_urls)>2:
      break;
print(new_urls)
len(new_urls)

{'https://www.foxnews.com/health/israeli-covid-19-drug-late-stage-coronavirus-treatment&sa=U&ved=2ahUKEwiCsrmgvK_0AhVkq3IEHdu_DH4Q0PADegQIAxAG&usg=AOvVaw33vw5aflwL_h6UWgkBrqo-', 'https://www.foxnews.com/health/us-covid-deaths-2021-surpass-2020-toll&sa=U&ved=2ahUKEwiCsrmgvK_0AhVkq3IEHdu_DH4Q0PADegQIAxAC&usg=AOvVaw1G1HYftKMvmgM2DEjVeAE9', 'https://www.foxnews.com/politics/psaki-attacks-desantis-over-disney-ending-vaccine-mandate&sa=U&ved=2ahUKEwiCsrmgvK_0AhVkq3IEHdu_DH4Q0PADegQIAxAK&usg=AOvVaw0QIMn7DTAiB13_g3-2NUJQ'}


3

##Below are the links scraped from automation code

In [29]:

page1 = "https://www.foxnews.com/health/israeli-covid-19-drug-late-stage-coronavirus-treatment"
page2 = "https://www.foxnews.com/health/us-covid-deaths-2021-surpass-2020-toll"
page3 = "https://www.foxnews.com/politics/psaki-attacks-desantis-over-disney-ending-vaccine-mandate"

output_file1 = "news1.txt"
output_file2 = "news2.txt"
output_file3 = "news3.txt"

pages = [page1,page2,page3]
output_files =[output_file1,output_file2,output_file3]

from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.request
import nltk

# https://www.nltk.org/book/ch05.html

In [30]:


nltk.download('punkt')


def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

for x in range(len(pages)):

    html = urllib.request.urlopen(pages[x]).read()
    page_text =text_from_html(html)

    my_list = nltk.tokenize.sent_tokenize(page_text) #Split into list of sentences.
 
    textfile = open(output_files[x], "w")

    for element in my_list:
        textfile.write(element + "\n")
    textfile.close()

    print(page_text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
   Fox News    U.S. Politics Media Opinion Business Entertainment Sports Lifestyle Weather TV Fox Nation Listen  More  Expand / Collapse search  Login  Watch TV  Menu  Hot Topics     Coronavirus  U.S.  Crime Military Education Terror Immigration Economy Personal Freedoms Fox News Investigates World  U.N. Conflicts Terrorism Disasters Global Economy Environment Religion Scandals Opinion  Politics  Executive Senate House Judiciary Foreign Policy Polls Elections Entertainment  Celebrity News Movies TV News Music News Style News Entertainment Video Business  Personal Finance Economy Markets Watchlist Lifestyle Real Estate Tech Lifestyle  Food + Drink Cars + Trucks Travel + Outdoors House + Home Fitness + Well-being Style + Beauty Family Faith Science  Archaeology Air & Space Planet Earth Wild Nature Natural Science Dinosaurs Tech  Security Innovation Drones Computers Video Games M

## Summarize text using transformers

In [4]:

!pip install transformers

     |████████████████████████████████| 3.1 MB 28.9 MB/s 
     |████████████████████████████████| 596 kB 43.2 MB/s 
     |████████████████████████████████| 895 kB 45.0 MB/s 
     |████████████████████████████████| 3.3 MB 47.7 MB/s 
     |████████████████████████████████| 59 kB 7.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [31]:
from transformers import pipeline
import os

In [6]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [32]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


In [33]:
summarizer = pipeline("summarization", model="t5-base", tokenizer="t5-base", framework="tf")

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [34]:
def pass_data(data_variable):
    new_data = data_variable.strip('“”')
    print (new_data)
    %%time
    print (len(new_data))
    text3= new_data[0:8000]
    summary_text = summarizer(text3, max_length=512, min_length=5, do_sample=False)[0]['summary_text']
    print(summary_text)
    return summary_text
    


In [35]:
# Import Module
import os


# Folder Path
path = "/content/"

# Change the directory
os.chdir(path)

# Read text File
def read_text_file(file_path):
    with open(file_path, 'r') as f:
        data = f.read().replace('\n', '')
        return pass_data(data)
         
  
  
# iterate through all file
summary_text_array=[]
for file in os.listdir():
    # Check whether file is in text format or not
    if file.endswith(".txt"):
        file_path = f"{path}{file}"
        # call read text file function
        summary_text_array.append(read_text_file(file_path))

        


Token indices sequence length is longer than the specified maximum sequence length for this model (1779 > 512). Running this sequence through the model will result in indexing errors


   Fox News    U.S.Politics Media Opinion Business Entertainment Sports Lifestyle Weather TV Fox Nation Listen  More  Expand / Collapse search  Login  Watch TV  Menu  Hot Topics     Coronavirus  U.S.  Crime Military Education Terror Immigration Economy Personal Freedoms Fox News Investigates World  U.N.Conflicts Terrorism Disasters Global Economy Environment Religion Scandals Opinion  Politics  Executive Senate House Judiciary Foreign Policy Polls Elections Entertainment  Celebrity News Movies TV News Music News Style News Entertainment Video Business  Personal Finance Economy Markets Watchlist Lifestyle Real Estate Tech Lifestyle  Food + Drink Cars + Trucks Travel + Outdoors House + Home Fitness + Well-being Style + Beauty Family Faith Science  Archaeology Air & Space Planet Earth Wild Nature Natural Science Dinosaurs Tech  Security Innovation Drones Computers Video Games Military Tech Health  Coronavirus Healthy Living Medical Research Mental Health Cancer Heart Health Children's Hea

In [36]:
len(summary_text_array)


6

## Distillation 1 - Parts of Speech

In [38]:
import nltk
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
for summary_text in summary_text_array:
  tokenized_text = word_tokenize(summary_text)
  pos_output = nltk.pos_tag(tokenized_text)
  print(pos_output)
  

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[('israeli', 'JJ'), ('biotechnology', 'NN'), ('company', 'NN'), ('Bonus', 'NNP'), ('BioGroup', 'NNP'), ('’', 'NNP'), ('s', 'NN'), ('cell', 'NN'), ('therapy', 'NN'), ('MesenCure', 'NNP'), ('was', 'VBD'), ('administered', 'VBN'), ('to', 'TO'), ('50', 'CD'), ('hospitalized', 'VBN'), ('COVID-19', 'JJ'), ('patients', 'NNS'), ('suffering', 'VBG'), ('from', 'IN'), ('life-threatening', 'JJ'), ('pneumonia', 'NN'), ('and', 'CC'), ('respiratory', 'NN'), ('distress', 'NN'), ('.', '.'), ('it', 'PRP'), ('found', 'VBD'), ('that', 'IN'), ('nearly', 'RB'), ('all', 'DT'), ('patients', 'NNS'), ('taking', 'VBG'), ('the', 'DT'), ('drug', 'NN'), ('survived', 'VBD'), ('.', '.'), ('bonus', 'NN'), ('biogroup', 'NN'), ('has', 'VBZ'), ('been', 'VBN'), ('seeking', 'VBG'), ('emergency', 'NN'), ('use', 'NN'), ('approval', 'NN'), (

## Distillation 2 - Named entity recognition

In [40]:

#spaCy is an NLP Framework -- easy to use and having ability to use neural networks
import en_core_web_sm
nlp = en_core_web_sm.load()
for summary_text in summary_text_array:
  doc = nlp(summary_text)
  print(doc.ents)
  print([(X.text, X.label_) for X in doc.ents])

(israeli, Bonus BioGroup, MesenCure, 50, bonus biogroup, Israel)
[('israeli', 'NORP'), ('Bonus BioGroup', 'ORG'), ('MesenCure', 'PRODUCT'), ('50', 'CARDINAL'), ('bonus biogroup', 'ORG'), ('Israel', 'GPE')]
(osha, 60%)
[('osha', 'ORG'), ('60%', 'PERCENT')]
(the u.s. department of health . ', Thanksgiving, this year)
[("the u.s. department of health . '", 'ORG'), ('Thanksgiving', 'DATE'), ('this year', 'DATE')]
(Thanksgiving, 1870, Christmas, New Year’s Day and Independence Day, 1863, Abraham Lincoln, Americans, the last Thursday)
[('Thanksgiving', 'DATE'), ('1870', 'DATE'), ('Christmas', 'DATE'), ('New Year’s Day and Independence Day', 'EVENT'), ('1863', 'DATE'), ('Abraham Lincoln', 'PERSON'), ('Americans', 'NORP'), ('the last Thursday', 'DATE')]
(more than 5 million, monday, u.s., more than 90,000, the tens of millions, four, at least six months, second)
[('more than 5 million', 'CARDINAL'), ('monday', 'DATE'), ('u.s.', 'GPE'), ('more than 90,000', 'CARDINAL'), ('the tens of millions',

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> Part 2: NLP POS Substitutions and Tones for Poets</h2>
</div>

#### Find 2 poems from 2 poets for example Robert Bly and Emily Dickison (One each shown below)


<li> Pick a poem from two poets, A, B
<li> Do two distillations including:  POS, embeddings
    POS do the following 
    do embeddings to find which poets are closest to each other in style
<li> Store each POS for each poet
<li> replace poet A POS with poet B  POS based in semantic similarity 

In [44]:
bly={
    'title':'And Now It’s September',
    'poem':'and the garden diminishes: cucumber leaves rumpled and rusty, zucchini felled by borers, tomatoes sparse on the vines. But out in the perennial beds, there’s one last blast of color: ignitions of goldenrod, flamboyant asters, spiraling mums, all those flashy spikes waving in the wind, conducting summer’s final notes. The ornamental grasses have gone to seed, haloed in the last light. Nights grow chilly, but the days are still warm; I wear the sun like a shawl on my neck and arms. Hundreds of blackbirds ribbon in, settle in the trees, so many black leaves, then, just as suddenly, they’re gone. This is autumn’s great Departure Gate, and everyone, boarding passes in hand, waits patiently in a long, long line.',
    'book': 'Stealing Sugar from the Castle: Selected and New Poems, 1950-2013 . W. W. Norton & Company. Kindle Edition. '
}


emily_dickinson={
    'title':'poem 48',
    'poem':'Sober for once, for what—for the words to budge. We spent summer propped up-by each others stuttering.There are seasons here if you squint. And theres relief in the landscapes sloughed off cusps of color fallen over the familiar landmarks, the familiar trash—things that last. ',
    'book':'JOSEPH MASSE'
}

# http://www.wisdomportal.com/Columba/EmilyDickinson-Dove.html
# https://www.poetryfoundation.org/poets/robert-bly

tokenized_text1 = word_tokenize(bly['poem'])
bly_pos= nltk.pos_tag(tokenized_text1)

tokenized_text2 = word_tokenize(emily_dickinson['poem'])
emily_pos= nltk.pos_tag(tokenized_text2)

In [15]:
print (bly_pos)

[('The', 'DT'), ('dove', 'NN'), ('returns', 'VBZ'), (';', ':'), ('it', 'PRP'), ('found', 'VBD'), ('no', 'DT'), ('resting', 'VBG'), ('place', 'NN'), (';', ':'), ('It', 'PRP'), ('was', 'VBD'), ('in', 'IN'), ('flight', 'NN'), ('all', 'DT'), ('night', 'NN'), ('above', 'IN'), ('the', 'DT'), ('shaken', 'JJ'), ('seas', 'NN'), ('.', '.'), ('Beneath', 'NNP'), ('Ark', 'NNP'), ('eaves', 'VBZ'), ('The', 'DT'), ('dove', 'NN'), ('shall', 'MD'), ('magnify', 'VB'), ('the', 'DT'), ('tiger', 'NN'), ('’', 'NNP'), ('s', 'NN'), ('bed', 'VBD'), (';', ':'), ('Give', 'VB'), ('the', 'DT'), ('dove', 'NN'), ('peace', 'NN'), ('.', '.'), ('The', 'DT'), ('split-tail', 'JJ'), ('swallows', 'NNS'), ('leave', 'VBP'), ('the', 'DT'), ('sill', 'NN'), ('at', 'IN'), ('dawn', 'NN'), (';', ':'), ('At', 'IN'), ('dusk', 'NN'), ('blue', 'NN'), ('swallows', 'VBZ'), ('shall', 'MD'), ('return', 'VB'), ('.', '.'), ('On', 'IN'), ('the', 'DT'), ('third', 'JJ'), ('day', 'NN'), ('the', 'DT'), ('crow', 'NN'), ('shall', 'MD'), ('fly', 'VB

In [16]:
print (emily_pos)

[('Once', 'RB'), ('more', 'JJR'), (',', ','), ('my', 'PRP$'), ('now', 'RB'), ('bewildered', 'VBN'), ('Dove', 'NNP'), (',', ','), ('Bestirs', 'NNP'), ('her', 'PRP$'), ('puzzled', 'JJ'), ('wings', 'NNS'), (',', ','), ('Once', 'RB'), ('more', 'RBR'), ('her', 'PRP$'), ('mistress', 'NN'), (',', ','), ('on', 'IN'), ('the', 'DT'), ('deep', 'JJ'), (',', ','), ('Her', 'NNP'), ('troubled', 'VBD'), ('question', 'NN'), ('flings—', 'NN'), (',', ','), ('Thrice', 'NNP'), ('to', 'TO'), ('the', 'DT'), ('floating', 'VBG'), ('casement', 'NN'), (',', ','), ('The', 'DT'), ('Patriarchs', 'NNP'), ('bird', 'NN'), ('returned', 'VBD'), (',', ','), ('Courage', 'NN'), ('!', '.'), ('My', 'PRP$'), ('brave', 'JJ'), ('Columba', 'NNP'), ('!', '.'), (',', ','), ('There', 'EX'), ('may', 'MD'), ('yet', 'RB'), ('be', 'VB'), ('Land', 'NN'), ('!', '.')]


## Swapping Verb Logic

In [45]:
def first_chunk_index(chunk, pred, start=0, step=1):
	
	l = len(chunk)
	end = l if step > 0 else -1
	
	for i in range(start, end, step):
		if pred(chunk[i]):
			return i
	
	return None

In [46]:
def swap_verb_phrase(chunk):
	'''Move modifier phrase after verb to front of chunk and drop the verb.
	>>> swap_verb_phrase([('the', 'DT'), ('book', 'NN'), ('was', 'VBD'), ('great', 'JJ')])
	[('great', 'JJ'), ('the', 'DT'), ('book', 'NN')]
	>>> swap_verb_phrase([('this', 'DT'), ('gripping', 'VBG'), ('book', 'NN'), ('is', 'VBZ'), ('fantastic', 'JJ')])
	[('fantastic', 'JJ'), ('this', 'DT'), ('gripping', 'VBG'), ('book', 'NN')]
	'''
	# find location of verb
	def vbpred(wt):
		word, tag = wt
		return tag != 'VBG' and tag.startswith('VB') and len(tag) > 2
	
	vbidx = first_chunk_index(chunk, vbpred)
	
	if vbidx is None:
		return chunk
	
	return chunk[vbidx+1:] + chunk[:vbidx]


In [47]:
swap_verb_phrase(bly_pos)

[('rumpled', 'VBN'),
 ('and', 'CC'),
 ('rusty', 'JJ'),
 (',', ','),
 ('zucchini', 'NN'),
 ('felled', 'VBN'),
 ('by', 'IN'),
 ('borers', 'NNS'),
 (',', ','),
 ('tomatoes', 'NNS'),
 ('sparse', 'VBP'),
 ('on', 'IN'),
 ('the', 'DT'),
 ('vines', 'NNS'),
 ('.', '.'),
 ('But', 'CC'),
 ('out', 'IN'),
 ('in', 'IN'),
 ('the', 'DT'),
 ('perennial', 'JJ'),
 ('beds', 'NNS'),
 (',', ','),
 ('there', 'EX'),
 ('’', 'VBZ'),
 ('s', 'NN'),
 ('one', 'CD'),
 ('last', 'JJ'),
 ('blast', 'NN'),
 ('of', 'IN'),
 ('color', 'NN'),
 (':', ':'),
 ('ignitions', 'NNS'),
 ('of', 'IN'),
 ('goldenrod', 'NN'),
 (',', ','),
 ('flamboyant', 'JJ'),
 ('asters', 'NNS'),
 (',', ','),
 ('spiraling', 'VBG'),
 ('mums', 'NNS'),
 (',', ','),
 ('all', 'PDT'),
 ('those', 'DT'),
 ('flashy', 'JJ'),
 ('spikes', 'NNS'),
 ('waving', 'VBG'),
 ('in', 'IN'),
 ('the', 'DT'),
 ('wind', 'NN'),
 (',', ','),
 ('conducting', 'VBG'),
 ('summer', 'NN'),
 ('’', 'NNP'),
 ('s', 'VBZ'),
 ('final', 'JJ'),
 ('notes', 'NNS'),
 ('.', '.'),
 ('The', 'DT'),
 

In [48]:
swap_verb_phrase(emily_pos)

[('summer', 'NN'),
 ('propped', 'VBN'),
 ('up-by', 'JJ'),
 ('each', 'DT'),
 ('others', 'NNS'),
 ('stuttering.There', 'EX'),
 ('are', 'VBP'),
 ('seasons', 'NNS'),
 ('here', 'RB'),
 ('if', 'IN'),
 ('you', 'PRP'),
 ('squint', 'VBP'),
 ('.', '.'),
 ('And', 'CC'),
 ('theres', 'VBZ'),
 ('relief', 'NN'),
 ('in', 'IN'),
 ('the', 'DT'),
 ('landscapes', 'NNS'),
 ('sloughed', 'VBD'),
 ('off', 'RP'),
 ('cusps', 'NN'),
 ('of', 'IN'),
 ('color', 'NN'),
 ('fallen', 'VBN'),
 ('over', 'IN'),
 ('the', 'DT'),
 ('familiar', 'JJ'),
 ('landmarks', 'NNS'),
 (',', ','),
 ('the', 'DT'),
 ('familiar', 'JJ'),
 ('trash—things', 'NNS'),
 ('that', 'IN'),
 ('last', 'JJ'),
 ('.', '.'),
 ('Sober', 'NNP'),
 ('for', 'IN'),
 ('once', 'RB'),
 (',', ','),
 ('for', 'IN'),
 ('what—for', 'PDT'),
 ('the', 'DT'),
 ('words', 'NNS'),
 ('to', 'TO'),
 ('budge', 'VB'),
 ('.', '.'),
 ('We', 'PRP')]

### References
[1] https://towardsdatascience.com/semantic-similarity-using-transformers-8f3cb5bf66d6

[2] https://towardsdatascience.com/abstractive-summarization-using-pytorch-f5063e67510

[3] https://towardsdatascience.com/topic-modeling-with-bert-779f7db187e6

[4] https://medium.com/@muddaprince456/categorizing-and-pos-tagging-with-nltk-python-28f2bc9312c3

[5] https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
